In [1]:
import os
import warnings
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

In [2]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing import image
from tensorflow.keras import models
from tensorflow.keras.models import load_model
from tensorflow.keras import models
from PIL import Image 
from skimage.io import imread
import cv2

K.clear_session()

# <b>IMAGE PROCESSSING</b>

In [ ]:
img = plt.imread('/Users/tushar/Documents/FINAL YEAR PROJECT/data/train/pizza/1008104.jpg.jpg')
dims = np.shape(img)
matrix = np.reshape(img, (dims[0] * dims[1], dims[2]))
print(np.shape(matrix))

In [ ]:
plt.imshow(img)
print("Image shape -> ",dims[:2])
print("Color channels -> ",dims[2])
print("Min color depth : {}, Max color depth {}".format(np.min(img),np.max(img)))

In [ ]:
from sklearn import cluster
n_vals=[2,4,6,8]
plt.figure(1, figsize=(12, 8))

for subplot,n in enumerate(n_vals):
    kmeans=cluster.KMeans(n)
    clustered = kmeans.fit_predict(matrix)
    dims = np.shape(img)
    clustered_img = np.reshape(clustered, (dims[0], dims[1]))
    plt.subplot(2,2, subplot+1)
    plt.title("n = {}".format(n), pad = 10,size=18)
    plt.imshow(clustered_img)
    
plt.tight_layout()

In [ ]:
bnorm = np.zeros_like(matrix, dtype=np.float32)
max_range = np.max(matrix, axis=1)
bnorm = matrix / np.vstack((max_range, max_range, max_range)).T
bnorm_img = np.reshape(bnorm, (dims[0],dims[1],dims[2]))
plt.figure(figsize=(8,10))
plt.imshow(bnorm_img)
plt.show()

 <h2> Sobel filter is a basic way to get an edge magnitude/gradient image. </h2>
    
**It works by calculating the gradient of image intensity at each pixel within the image. It finds the direction of the largest increase from light to dark and the rate of change in that direction.**

In [ ]:
import skimage
from skimage.feature import greycomatrix, greycoprops
from skimage.filters import sobel
from skimage.filters import sobel_h

plt.figure(1,figsize=(20,15))
cmap="YlGnBu"
plt.subplot(3,1,1)
plt.imshow(img)

plt.subplot(3,1,2)
plt.imshow(sobel(img[:,:,2]),cmap=cmap)

plt.subplot(3,1,3)
plt.imshow(sobel_h(img[:,:,1]), cmap=cmap)

plt.tight_layout()

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(3)
pca.fit(matrix)
img_pca = pca.transform(matrix)
img_pca = np.reshape(img_pca, (dims[0], dims[1], dims[2]))

fig = plt.figure(figsize=(8, 8))
plt.imshow(img_pca[:,:,1], cmap=cmap)

In [ ]:
main='/Users/tushar/Documents/FINAL YEAR PROJECT/data/train'

data=dict()

for i in os.listdir(main):
    sub_dir=os.path.join(main,i)
    count=len(os.listdir(sub_dir))
    data[i]=count
    
  
keys = data.keys()
values = data.values()

colors=["red" if x<= 150 else "green" for x in values]

fig, ax = plt.subplots(figsize=(12,8))
y_pos=np.arange(len(values))
plt.barh(y_pos,values,align='center',color=colors)
for i, v in enumerate(values):
    ax.text(v+1.4, i-0.25, str(v), color=colors[i])
ax.set_yticks(y_pos)
ax.set_yticklabels(keys)
ax.set_xlabel('Images',fontsize=16)
plt.xticks(color='black',fontsize=13)
plt.yticks(fontsize=13)
plt.show()

## Let's visualize our dataset by randomly picking an image from every class

In [ ]:
import random

train_folder = "/Users/tushar/Documents/FINAL YEAR PROJECT/data/train"
images = []

for food_folder in sorted(os.listdir(train_folder)):
    food_items = os.listdir(train_folder + '/' + food_folder)
    food_selected = np.random.choice(food_items)
    images.append(os.path.join(train_folder,food_folder,food_selected))
                                     
fig=plt.figure(1, figsize=(25, 25))

for subplot,image_ in enumerate(images):
    category=image_.split('/')[-2]
    imgs = plt.imread(image_)
    a,b,c=imgs.shape
    fig=plt.subplot(5, 4, subplot+1)
    fig.set_title(category, pad = 10,size=18)
    plt.imshow(imgs)
    
plt.tight_layout()

# <b> MODEL TRAINING </b>

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

n_classes = 20
batch_size = 30
img_width, img_height = 299, 299

train_data_dir = '/Users/tushar/Documents/FINAL YEAR PROJECT/data/train'

# Data Augmentation with ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=40,  # Increased rotation range
    width_shift_range=0.2,  # Added width shift
    height_shift_range=0.2,  # Added height shift
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

val_data_dir = '/Users/tushar/Documents/FINAL YEAR PROJECT/data/val'

val_datagen = ImageDataGenerator(rescale=1. / 255)

val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
class_map = train_generator.class_indices
class_map

## Training the model

## Saving the model

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras import regularizers
import math

nb_train_samples = 15000 
nb_validation_samples = 2500

# Load MobileNetV2 with pre-trained weights
mobilenet = MobileNetV2(weights='imagenet', include_top=False)

# Fine-tune the last few layers of MobileNetV2
for layer in mobilenet.layers[:-50]:  # Fine-tune more layers
    layer.trainable = False
    
# Add your custom layers for classification
x = mobilenet.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu', kernel_regularizer='l2')(x)  # Increased units and added L2 regularization
x = Dropout(0.5)(x)

predictions = Dense(n_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=mobilenet.input, outputs=predictions)

# Implement Learning Rate Scheduler
def lr_scheduler(epoch):
    initial_lr = 0.001
    drop = 0.5
    epochs_drop = 10.0
    lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lr

lr_callback = LearningRateScheduler(lr_scheduler)

# Compile the model with a lower initial learning rate
model.compile(optimizer=SGD(lr=0.0005, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# Set up callbacks
checkpointer = ModelCheckpoint(filepath='v3_mobilenet', verbose=1, save_best_only=True)
csv_logger = CSVLogger('history_v3_mobilenet.log')

# Train the model
history = model.fit_generator(train_generator,
                              steps_per_epoch=len(train_generator),
                              validation_data=val_generator,
                              validation_steps=len(val_generator),
                              epochs=100,
                              verbose=1,
                              callbacks=[csv_logger, checkpointer, lr_callback])

In [14]:
 model.save('model_v3_mobilenet.h5')

## Accuracy and Loss curves

In [ ]:
def plot_accuracy(history):
    
    plt.plot(history.history['accuracy'],label='train accuracy')
    plt.plot(history.history['val_accuracy'],label='validation accuracy')
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='best')
    plt.savefig('Accuracy_v3_mobilenet')
    plt.show()
    
def plot_loss(history):
    
    plt.plot(history.history['loss'],label="train loss")
    plt.plot(history.history['val_loss'],label="validation loss")
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='best')
    plt.savefig('Loss_v3_mobilenet')
    plt.show()
    
plot_accuracy(history)
plot_loss(history)

# <b> PREDICTIONS </b>

## Load the model

In [ ]:
K.clear_session()
path_to_model='./model_v3_mobilenet.h5'
print("Loading the model..")
model = load_model(path_to_model)
print("Done!")

## Testing model on test set 

In [ ]:
test_data_dir = '/Users/tushar/Documents/FINAL YEAR PROJECT/data/test'

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
scores = model.evaluate_generator(test_generator)

print("Test Accuracy: {:.3f}".format(scores[1]))

## Function to predict single image or predict all images from a directory

- <h3> Single image prediction </h3>

In [19]:
category={
    0: ['apple_pie','Apple Pie'], 1: ['cannoli','Cannoli'], 2: ['chicken_curry','Chicken Curry'],
    3: ['chocolate_cake','Chocolate Cake'], 4: ['cup_cake','Cup Cake'], 5: ['donuts','Donuts'],
    6: ['dumplings','Dumplings'], 7: ['french_fries','French Fries'], 8: ['fried_rice','Fried Rice'], 9: ['hamburger','Hamburger'],
    10: ['hot_and_sour_soup','Hot and Sour Soup'], 11: ['hot_dog','Hot Dog'], 12: ['ice_cream','Ice Cream'],
    13: ['nachos','Nachos'], 14: ['omlette','Omlette'], 15: ['pizza','Pizza'],
    16: ['ramen','Ramen'], 17: ['samosa','Samosa'], 18: ['spring_rolls','Spring Rolls'], 19: ['waffles','Waffles']
}

def predict_image(filename,model):
    img_ = image.load_img(filename, target_size=(299, 299))
    img_array = image.img_to_array(img_)
    img_processed = np.expand_dims(img_array, axis=0) 
    img_processed /= 255.   
    
    prediction = model.predict(img_processed)
    
    index = np.argmax(prediction)
    
    plt.title("Prediction - {}".format(category[index][1]))
    plt.imshow(img_array)
    
def predict_dir(filedir,model):
    cols=5
    pos=0
    images=[]
    total_images=len(os.listdir(filedir))
    rows=total_images//cols + 1
    
    true=filedir.split('/')[-1]
    
    fig=plt.figure(1, figsize=(25, 25))
    
    for i in sorted(os.listdir(filedir)):
        images.append(os.path.join(filedir,i))
        
    for subplot,imggg in enumerate(images):
        img_ = image.load_img(imggg, target_size=(299, 299))
        img_array = image.img_to_array(img_)
        
        img_processed = np.expand_dims(img_array, axis=0) 

        img_processed /= 255.
        prediction = model.predict(img_processed)
        index = np.argmax(prediction)
        
        pred=category.get(index)[0]
        if pred==true:
            pos+=1
        
        fig=plt.subplot(rows, cols, subplot+1)
        fig.set_title(category.get(index)[1], pad = 10,size=18)
        plt.imshow(img_array)

    acc=pos/total_images
    print("Accuracy of Test : {:.2f} ({pos}/{total})".format(acc,pos=pos,total=total_images))
    plt.tight_layout()

In [ ]:
predict_image('/Users/tushar/Documents/FINAL YEAR PROJECT/data/test/hamburger/1007277.jpg',model)

- <h3> Predicting category </h3>

In [ ]:
predict_dir("/Users/tushar/Documents/FINAL YEAR PROJECT/data/test/hamburger",model)

## Let's plot a confusion matrix for all the food items

In [22]:
from sklearn.metrics import confusion_matrix
import itertools

img_width, img_height = 299, 299

def labels_confusion_matrix():
    folder_path="/Users/tushar/Documents/FINAL YEAR PROJECT/data/test"
    
    mapping={}
    for i,j in enumerate(sorted(os.listdir(folder_path))):
        mapping[j]=i
    
    files=[]
    real=[]
    predicted=[]

    for i in os.listdir(folder_path):
        
        true=os.path.join(folder_path,i)
        true=true.split('/')[-1]
        true=mapping[true]
        
        for j in os.listdir(os.path.join(folder_path,i)):
            
            img_ = image.load_img(os.path.join(folder_path,i,j), target_size=(img_height, img_width))
            img_array = image.img_to_array(img_)
            img_processed = np.expand_dims(img_array, axis=0) 
            img_processed /= 255.
            prediction = model.predict(img_processed)
            index = np.argmax(prediction)

            predicted.append(index)
            real.append(true)
            
    return (real,predicted)

def print_confusion_matrix(real,predicted):

    cmap="viridis"
    cm_plot_labels = [i for i in range(20)]

    cm = confusion_matrix(y_true=real, y_pred=predicted)
    df_cm = pd.DataFrame(cm,cm_plot_labels,cm_plot_labels)
    sns.set(font_scale=1.1) # for label size
    plt.figure(figsize = (15,10))
    s=sns.heatmap(df_cm, annot=True,cmap=cmap) # font size
#     bottom,top=s.get_ylim()
#     s.set_ylim(bottom+0.6,top-0.6)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig('confusion_matrix.png')
    plt.show()

In [ ]:
y_true,y_pred=labels_confusion_matrix()
print_confusion_matrix(y_true,y_pred)

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import os

category={
    0: ['apple_pie','Apple Pie'], 1: ['cannoli','Cannoli'], 2: ['chicken_curry','Chicken Curry'],
    3: ['chocolate_cake','Chocolate Cake'], 4: ['cup_cakes','Cup Cakes'], 5: ['donuts','Donuts'],
    6: ['dumplings','Dumplings'], 7: ['french_fries','French Fries'], 8: ['fried_rice','Fried Rice'], 9: ['hamburger','Hamburger'],
    10: ['hot_and_sour_soup','Hot and Sour Soup'], 11: ['hot_dog','Hot Dog'], 12: ['ice_cream','Ice Cream'],
    13: ['nachos','Nachos'], 14: ['omelette','Omelette'], 15: ['pizza','Pizza'],
    16: ['ramen','Ramen'], 17: ['samosa','Samosa'], 18: ['spring_rolls','Spring Rolls'], 19: ['waffles','Waffles']
}

def load_images_and_labels(data_dir):
    images, labels = [], []
    for i, category_info in category.items():
        category_name = category_info[0]
        category_path = os.path.join(data_dir, category_name)
        
        for img_file in os.listdir(category_path):
            img_path = os.path.join(category_path, img_file)
            images.append(img_path)
            labels.append(i)
    
    return images, labels


# Load images and labels from the testing set
test_data_dir = '/Users/tushar/Documents/FINAL YEAR PROJECT/data/test'
test_images, test_labels = load_images_and_labels(test_data_dir)

# Binarize the labels
labels_bin = label_binarize(test_labels, classes=list(category.keys()))

# Initialize an empty array to store predictions
predictions = []

# Make predictions for each image in the testing set
for img_path in test_images:
    img = image.load_img(img_path, target_size=(299, 299))
    img_array = image.img_to_array(img)
    img_processed = np.expand_dims(img_array, axis=0)
    img_processed /= 255.
    
    prediction = model.predict(img_processed)
    predictions.append(prediction.flatten())

y_scores = np.array(predictions)

# Compute ROC curve and AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(len(category)):
    fpr[i], tpr[i], _ = roc_curve(labels_bin[:, i], y_scores[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve for each class
plt.figure(figsize=(8, 8))
for i in range(len(category)):
    plt.plot(fpr[i], tpr[i], lw=2, label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('AUC-ROC Curve for Food Classification')
plt.legend(loc='lower right')
plt.show()

In [ ]:
from sklearn.metrics import classification_report
import time
import numpy as np

print(classification_report(y_true,y_pred))

<h1> <b> MODEL LEARNING VISUALIZATIONS </b></h1>

### SOME HELPER FUNCTIONS WHICH WILL ENABLE US TO VISUALIZE HOW NEURAL NETWORK WORKS AND PERFORMS!

In [26]:
def get_activations(img, model_activations):
    img = image.load_img(img, target_size=(299, 299))
    img = image.img_to_array(img)                    
    img = np.expand_dims(img, axis=0)         
    img /= 255. 
    plt.imshow(img[0])
    plt.show()
    return model_activations.predict(img)

def show_activations(activations, layer_names):
    
    images_per_row = 16

    # Now let's display our feature maps
    for layer_name, layer_activation in zip(layer_names, activations):
        # This is the number of features in the feature map
        n_features = layer_activation.shape[-1]

        # The feature map has shape (1, size, size, n_features)
        size = layer_activation.shape[1]

        # We will tile the activation channels in this matrix
        n_cols = n_features // images_per_row
        display_grid = np.zeros((size * n_cols, images_per_row * size))

        # We'll tile each filter into this big horizontal grid
        for col in range(n_cols):
            for row in range(images_per_row):
                channel_image = layer_activation[0,:, :,col * images_per_row + row]
                # Post-process the feature to make it visually palatable
                channel_image -= channel_image.mean()
                channel_image /= channel_image.std()
                channel_image *= 64
                channel_image += 128
                channel_image = np.clip(channel_image, 0, 255).astype('uint8')
                display_grid[col * size : (col + 1) * size,row * size : (row + 1) * size] = channel_image

        # Display the grid
        scale = 1. / size
        plt.figure(figsize=(scale * display_grid.shape[1],
                            scale * display_grid.shape[0]))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='viridis')

    plt.show()
    
def activation_conv():
    first_convlayer_activation = activations[0]
    second_convlayer_activation = activations[3]
    third_convlayer_activation = activations[6]
    f,ax = plt.subplots(1,3, figsize=(10,10))
    ax[0].imshow(first_convlayer_activation[0, :, :, 3], cmap='viridis')
    ax[0].axis('OFF')
    ax[0].set_title('Conv2d_1')
    ax[1].imshow(second_convlayer_activation[0, :, :, 3], cmap='viridis')
    ax[1].axis('OFF')
    ax[1].set_title('Conv2d_2')
    ax[2].imshow(third_convlayer_activation[0, :, :, 3], cmap='viridis')
    ax[2].axis('OFF')
    ax[2].set_title('Conv2d_3')
    
    
def get_attribution(food):
    
    tf.compat.v1.disable_eager_execution()
    
    img = image.load_img(food, target_size=(299, 299))
    img = image.img_to_array(img) 
    img /= 255. 
    f,ax = plt.subplots(1,3, figsize=(15,15))
    ax[0].imshow(img)
    
    img = np.expand_dims(img, axis=0)
    model = load_model('./model_v1_inceptionV3.h5')
        
    preds = model.predict(img)
    class_id = np.argmax(preds[0])
    ax[0].set_title("Input Image")
    class_output = model.output[:, class_id]
    last_conv_layer = model.get_layer("mixed10")
    
    grads = K.gradients(class_output, last_conv_layer.output)[0]
    pooled_grads = K.mean(grads, axis=(0, 1, 2))
    iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
    pooled_grads_value, conv_layer_output_value = iterate([img])
    for i in range(2048):
        conv_layer_output_value[:, :, i] *= pooled_grads_value[i]
    
    heatmap = np.mean(conv_layer_output_value, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    ax[1].imshow(heatmap)
    ax[1].set_title("Heat map")
    
    
    act_img = cv2.imread(food)
    heatmap = cv2.resize(heatmap, (act_img.shape[1], act_img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed = cv2.addWeighted(act_img, 0.6, heatmap, 0.4, 0)
    cv2.imwrite('classactivation.png', superimposed)
    img_act = image.load_img('classactivation.png', target_size=(299, 299))
    ax[2].imshow(img_act)
    ax[2].set_title("Class Activation")
    plt.show()
    return preds

## MODEL LAYERS 

In [ ]:
print("Total layers in the model : ",len(model.layers),"\n")

# We start with index 1 instead of 0, as input layer is at index 0
layers = [layer.output for layer in model.layers[1:11]]
# We now initialize a model which takes an input and outputs the above chosen layers
activations_output = models.Model(inputs=model.input, outputs=layers)
# print(layers)

layer_names = []
for layer in model.layers[1:11]: 
    layer_names.append(layer.name)
    
print("First 10 layers which we can visualize are -> ", layer_names)


# <b>LAYER WISE ACTIVATIONS</b>

In [ ]:
food = '/kaggle/input/tushar/val/val/pizza/1220156.jpg'
activations = get_activations(food,activations_output)
show_activations(activations, layer_names)

In [ ]:
activation_conv()

## This time let's visualize some other food item's layer.

In [ ]:
food = '/kaggle/input/tushar/val/val/pizza/1220156.jpg'
activations = get_activations(food,activations_output)
show_activations(activations, layer_names)

In [ ]:
activation_conv()

# <b>GENERATING HEATMAPS</b>

In [ ]:
pred = get_attribution('/kaggle/input/tushar/val/val/samosa/107871.jpg')

In [ ]:
pred2=get_attribution('/kaggle/input/tushar/val/val/fried_rice/1084.jpg')

In [ ]:
pred3=get_attribution('/kaggle/input/tushar/val/val/omelette/1145379.jpg')

In [ ]:
pred4=get_attribution('/kaggle/input/tushar/val/val/chicken_curry/1110023.jpg')

In [ ]:
pred5=get_attribution('/kaggle/input/tushar/val/val/apple_pie/123782.jpg')

## Downloading random image from net to predict and generate heatmap

In [ ]:
!wget -O download.jpg https://www.cookwithmanali.com/wp-content/uploads/2015/01/Restaurant-Style-Dal-Makhani-Recipe.jpg
    
model_load = load_model('./model_v1_inceptionV3.h5')

In [ ]:
pred = get_attribution('download.jpg')